https://earthquake.usgs.gov/earthquakes/search/

In [1]:
from datetime import datetime

from parser.app import CsvParser

In [2]:
start = datetime(2020,3,6)
end = datetime(2023,6,15)


parser = CsvParser(start, end, 'Russia_2000-2023')
parser.to_csv()

strat (datetime.datetime(2000, 3, 6, 0, 0), datetime.datetime(2000, 12, 31, 0, 0)) period
strat (datetime.datetime(2001, 1, 1, 0, 0), datetime.datetime(2001, 12, 31, 0, 0)) period
strat (datetime.datetime(2002, 1, 1, 0, 0), datetime.datetime(2002, 12, 31, 0, 0)) period
strat (datetime.datetime(2003, 1, 1, 0, 0), datetime.datetime(2003, 12, 31, 0, 0)) period
strat (datetime.datetime(2004, 1, 1, 0, 0), datetime.datetime(2004, 12, 31, 0, 0)) period
strat (datetime.datetime(2005, 1, 1, 0, 0), datetime.datetime(2005, 12, 31, 0, 0)) period
strat (datetime.datetime(2006, 1, 1, 0, 0), datetime.datetime(2006, 12, 31, 0, 0)) period
strat (datetime.datetime(2007, 1, 1, 0, 0), datetime.datetime(2007, 12, 31, 0, 0)) period
strat (datetime.datetime(2008, 1, 1, 0, 0), datetime.datetime(2008, 12, 31, 0, 0)) period
strat (datetime.datetime(2009, 1, 1, 0, 0), datetime.datetime(2009, 12, 31, 0, 0)) period
strat (datetime.datetime(2010, 1, 1, 0, 0), datetime.datetime(2010, 12, 31, 0, 0)) period
strat (dat